In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
import tensorflow_addons
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2

In [23]:
def get_count(dir_path):    
    for path in os.listdir(dir_path):
        count = 0
        for item in os.listdir(dir_path + '\\' + path):
            count += 1
        print(path, 'Images count:', count)


print('Training data:')
get_count('C:\\Users\\sansk\\Desktop\\archive\\TRAINING_CG-1050\\TRAINING')

print('\nValidation data:')
get_count('C:\\Users\\sansk\\Desktop\\archive\\VALIDATION_CG-1050\\VALIDATION')

Training data:
ORIGINAL Images count: 730
TAMPERED Images count: 730

Validation data:
ORIGINAL Images count: 314
TAMPERED Images count: 314


In [ ]:
img_shape = (150,150)

datagen = ImageDataGenerator()

training_data = datagen.flow_from_directory('C:\\Users\\sansk\\Desktop\\archive\\TRAINING_CG-1050\\TRAINING',
                                            target_size=img_shape,
                                            color_mode='rgb',
                                            batch_size=64,
                                            seed=32,
                                            interpolation='bicubic')

validation_data = datagen.flow_from_directory('C:\\Users\\sansk\\Desktop\\archive\\VALIDATION_CG-1050\\VALIDATION',
                                                    target_size=img_shape,
                                                    color_mode='rgb',
                                                    batch_size=64,
                                                    seed=32,
                                                    interpolation='bicubic')

In [ ]:
keras = tf.keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides=2, activation="relu", input_shape=(150, 150, 3)))
model.add(tensorflow_addons.layers.InstanceNormalization(axis=-1))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=2))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', strides=2))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, 'relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(2, 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [22]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2)
model.fit(training_data, validation_data=validation_data, epochs=40, callbacks=[early_stop, reduce_lr])
# model.fit(training_data, validation_data=validation_data, epochs=3)

Epoch 1/40
23/23 [==============================] - 115s 5s/step - loss: 0.6243 - accuracy: 0.6438 - val_loss: 0.6342 - val_accuracy: 0.6465 - lr: 1.0000e-06
Epoch 2/40
23/23 [==============================] - 100s 4s/step - loss: 0.6257 - accuracy: 0.6466 - val_loss: 0.6342 - val_accuracy: 0.6449 - lr: 1.0000e-06
Epoch 3/40
23/23 [==============================] - 111s 5s/step - loss: 0.6258 - accuracy: 0.6459 - val_loss: 0.6342 - val_accuracy: 0.6449 - lr: 1.0000e-06
Epoch 4/40
23/23 [==============================] - 93s 4s/step - loss: 0.6226 - accuracy: 0.6452 - val_loss: 0.6342 - val_accuracy: 0.6449 - lr: 1.0000e-07
Epoch 5/40
23/23 [==============================] - 96s 4s/step - loss: 0.6190 - accuracy: 0.6452 - val_loss: 0.6342 - val_accuracy: 0.6449 - lr: 1.0000e-07


In [ ]:
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
acc = model.history.history['accuracy']
val_acc = model.history.history['val_accuracy']
epochs = [i for i in range(len(loss))]
plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.show()

In [ ]:
plt.plot(epochs,acc)
plt.plot(epochs,val_acc)
plt.show()

In [ ]:
def predict_image(path):
    image=cv2.imread(path)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    image=cv2.resize(image,(150,150))
    image=image.reshape(1,150,150,3)
    prediction=np.argmax(model.predict(image))
    labels=['Original','Tampered']
    return labels[prediction]

In [ ]:
predict_image('C:\\Users\\sansk\\Desktop\\archive\\TRAINING_CG-1050\\TRAINING\\ORIGINAL\\Im100_2_cm.jpg')

In [ ]:
predict_image('C:\\Users\\sansk\\Downloads\\a.jpg')